In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="mlops-aml-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: mlops
Azure region: eastus2
Subscription id: 4238cf71-2de7-41b0-a073-09f11ffcccea
Resource group: mlops


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "ComputeCluster"

# Verifing that cluster does not exist already or Create a New Cluster
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster.')
except ComputeTargetException:
    print('Creating a new compute cluster....')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# get_status() to get a detailed status for the current running cluster 
print(compute_target.get_status().serialize())

Found existing cluster.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-09T12:55:19.034000+00:00', 'errors': None, 'creationTime': '2021-02-09T12:55:15.684450+00:00', 'modifiedTime': '2021-02-09T12:55:31.491033+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
import os
from random import choice

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "C": uniform(0, 10),
        "max_iter": quniform(20, 180, 1)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.get_details()
hyperdrive_run.get_children_sorted_by_primary_metric(top=1)


[{'run_id': 'HD_0b1cf3e5-c373-4fd5-80b6-92587083a92e_4',
  'hyperparameters': '{"C": 5.09070672763789, "max_iter": 123.0}',
  'best_primary_metric': 0.9129742033383915,
  'status': 'Completed'}]

In [19]:
#Save the best model

best_run.get_file_names()


['azureml-logs/55_azureml-execution-tvmps_e9757d638c22cbb3532b59fb7acb7f7ac2fc825ea2b02e88b53073d1453857c8_d.txt',
 'azureml-logs/65_job_prep-tvmps_e9757d638c22cbb3532b59fb7acb7f7ac2fc825ea2b02e88b53073d1453857c8_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_e9757d638c22cbb3532b59fb7acb7f7ac2fc825ea2b02e88b53073d1453857c8_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/106_azureml.log',
 'logs/azureml/dataprep/python_span_01eaf517-ee1f-43e7-8bf4-5a8b21f54156.jsonl',
 'logs/azureml/dataprep/python_span_02dc5a5d-c4c7-4e2a-a912-49b8410147c2.jsonl',
 'logs/azureml/dataprep/python_span_08c4feca-6afa-4da4-a34a-c5a8996ddab4.jsonl',
 'logs/azureml/dataprep/python_span_0a1fb867-7eef-41e7-9f78-41b81922ea61.jsonl',
 'logs/azureml/dataprep/python_span_0cc1ee52-4f8d-42d4-9af3-f527aac5d1d0.jsonl',
 'logs/azureml/dataprep/python_span_1308a33c-8860-466c-969f-b765b5fc3ea4.jsonl',
 'logs/azureml/dataprep/python_span_1a312be3-c991-4f55-92af-2d984ff0b1c3.jsonl',
 'logs/azureml/dataprep/python_span_1a5bdff0-d138-4a7f-9ce9-5ddb509c7319.jsonl',
 'logs/azureml/dataprep/python_span_1ec59a1e-0ab0-4afd-b6c1-a8197d17d7e0.jsonl',
 'logs/azureml/dataprep/python_span_200da058-4fd6-4e1b-8d54-437459248d3e.jsonl',
 'logs/azureml/dataprep/python_span_2267cbba-c57c-4041-9266-7c4c799c26f8.jsonl',
 'logs/azureml/dataprep/python_span_27c0aaa9-d362-4b48-a97d-a830eacd6977.jsonl',
 'logs/azureml/dataprep/python_span_27c51c7b-7afd-4150-a922-f2a029b4b64d.jsonl',
 'logs/azureml/dataprep/python_span_2acd702d-42bb-45a6-9c4e-5286058fb639.jsonl',
 'logs/azureml/dataprep/python_span_2c101ebc-84a3-4a00-af26-5433183ba4e4.jsonl',
 'logs/azureml/dataprep/python_span_2c9314ec-63cb-4cc5-b39b-227589bd1d33.jsonl',
 'logs/azureml/dataprep/python_span_3123ce5e-d1df-4920-8e94-ad51047cef46.jsonl',
 'logs/azureml/dataprep/python_span_315da429-081f-48f2-a33c-d9e20c629e72.jsonl',
 'logs/azureml/dataprep/python_span_333e60b5-df80-43ae-9041-a95604c79c9e.jsonl',
 'logs/azureml/dataprep/python_span_38d6163b-a49f-4916-a032-75561032d5f2.jsonl',
 'logs/azureml/dataprep/python_span_39cac017-0939-4ab6-98c2-f05513445814.jsonl',
 'logs/azureml/dataprep/python_span_3b709c88-d290-47ae-939e-df16b4922109.jsonl',
 'logs/azureml/dataprep/python_span_4259a60a-2114-4c62-acc0-f025dab6b1e4.jsonl',
 'logs/azureml/dataprep/python_span_43050b1e-8d17-4290-a107-496a8f6ec1c7.jsonl',
 'logs/azureml/dataprep/python_span_47dd7298-af89-403d-854e-2abb6dfddfae.jsonl',
 'logs/azureml/dataprep/python_span_4a684d28-858f-4983-aafc-379a12dd14b2.jsonl',
 'logs/azureml/dataprep/python_span_4df5dd91-f05a-4857-9492-35f70c6e0694.jsonl',
 'logs/azureml/dataprep/python_span_502bfbe7-8d84-4080-b80b-88257d57381a.jsonl',
 'logs/azureml/dataprep/python_span_521d52f9-d4d4-4c29-8c05-afe0e012582a.jsonl',
 'logs/azureml/dataprep/python_span_52442d55-b697-42e3-819b-6fafc3dd407d.jsonl',
 'logs/azureml/dataprep/python_span_52eef2c7-aa47-4440-a970-d00c45c45e76.jsonl',
 'logs/azureml/dataprep/python_span_554638b2-bdab-4907-8e19-71601455e588.jsonl',
 'logs/azureml/dataprep/python_span_56f3c3c5-a8bc-4958-a008-7f0daf6ba0f6.jsonl',
 'logs/azureml/dataprep/python_span_5f421296-840a-4324-9b75-8869f457b82d.jsonl',
 'logs/azureml/dataprep/python_span_61a59ed8-094a-4506-9735-a5999d491fe4.jsonl',
 'logs/azureml/dataprep/python_span_635b3df1-6ed6-47bd-b315-8d828050b092.jsonl',
 'logs/azureml/dataprep/python_span_666082a9-8c1f-4781-afec-456958438925.jsonl',
 'logs/azureml/dataprep/python_span_6a54ca2c-2d2c-4538-abb5-d947b1594e09.jsonl',
 'logs/azureml/dataprep/python_span_744d7123-9c12-4373-8ab2-1781515d1195.jsonl',
 'logs/azureml/dataprep/python_span_74d2884a-a98f-4c35-97d3-9d680ab7f9f6.jsonl',
 'logs/azureml/dataprep/python_span_7c0bbdf8-992b-4c1d-ba59-833089cf99e0.jsonl',
 'logs/azureml/dataprep/python_span_7f1bfc06-fd31-461f-96e5-08e8d489e0a2.jsonl',
 'logs/azureml/dataprep/python_span_81fa3071-f363-4e8f-bb63-89992e882a4f.jsonl',
 'logs/azureml/dataprep/python_span_836f39a7-da5f-

In [23]:
#Register the model

#model = best_run.register_model(model_name="hyperdrive_model",model_path="/model.joblib")

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

import pandas as pd
data = pd.concat([x,y], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=50,
    task="classification",
    primary_metric="accuracy",
    training_data=data,
    label_column_name='y',
    n_cross_validations=5,
    )

In [12]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_91a42968-cac3-4d5a-a8dd-8a3d727639d7

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
****************************************************************************************************


In [13]:
# Retrieve and save your best automl model.

best_run

Experiment,Id,Type,Status,Details Page,Docs Page
mlops-aml-project,HD_0b1cf3e5-c373-4fd5-80b6-92587083a92e_4,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#Save the model
joblib.dump(best_run, 'hd_bestrun_model.joblib')
